In [1]:
from datetime import datetime
import requests
import numpy as np
import pandas as pd
import yaml
import re
from bs4 import BeautifulSoup
import wikitextparser as wtp
from ratelimit import rate_limited
from operator import *

def save_obj(obj, name ):
    import pickle
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    import pickle
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def easyText(t):
    return re.sub(r'\W+', '', str(t)).lower()

In [2]:
r = requests.get(url='http://liquipedia.net/dota2/api.php?action=ask&query=[[%3A%2B]]+[[Has+teamid%3A%3A>0]]|%3FHas+id|%3FHas+id+lc|%3FHas+location|sort%3DHas+id|limit%3D500&format=json')
json2 = r.json()

In [109]:
df

,name,id,id lc,location
0,(monkey) Business,(monkey) Business,(monkey) business,Europe
1,(monkey) Business,(monkey) Business,(monkey) business,Europe
2,.aSpera.,.aSpera.,aspera,Russia
3,1st.VN,1st.VN,1st.vn,Vietnam
4,4 Anchors + Sea Captain,4 Anchors + Sea Captain,4 anchors + sea captain,Finland
5,4 Clover & Lepricon,4 Clover & Lepricon,4 clover & lepricon,Europe
6,4 Friends + Chrillee,4 Friends + Chrillee,4 friends + chrillee,Sweden
7,4 Vikings + Dane,4 Vikings + Dane,4 vikings + dane,Sweden
8,4 protect five,4 protect five,4 protect five,Sweden
9,5Jungz,5Jungz,5jungz,Europe


In [14]:
df = pd.DataFrame(columns=['name', 'id', 'id lc', 'location'])
idlcs = []

In [16]:
for lname, ldata in json2.get('query').get('results').items():
    po = ldata.get('printouts')
    pid = ldata.get('fulltext')
    try:
        idn = po.get('Has id')[0]
    except IndexError:
        idn = 'Nan' 
    try:
        idlc = po.get('Has id lc')[0]
    except IndexError:
        idlc = 'Nan'
    try:
        loc = po.get('Has location')[0]
    except IndexError:
        loc = 'Nan'        
    df = df.append({'name': lname, 'id': idn, 'id lc': idlc, 'location': loc}, ignore_index=True)
    idlcs.append(idlc)

In [17]:
idlcs

['(monkey) business',
 'aspera',
 '1st.vn',
 '4 anchors + sea captain',
 '4 clover & lepricon',
 '4 friends + chrillee',
 '4 vikings + dane',
 '4 protect five',
 '5jungz',
 '90s gaming',
 'alternate attax',
 'aces gaming',
 'acion arena',
 'ad finem',
 'alliance',
 'ancient warriors',
 'animal planet',
 'arcade.powerrangers',
 'arcanys gaming',
 'avalon club',
 'b)ears',
 'boom id',
 'balrogs',
 'basically unknown',
 'beyond infinity',
 'big god',
 'big red machine',
 'bravado gaming',
 'brave heart',
 'broodmothers',
 'burden united',
 'cavalry',
 'cdec avenger',
 'cdec gaming',
 'cdec youth',
 'cis rejects',
 'cookiss',
 "can't say wips",
 'cascade',
 "summer's rift",
 "click 'n search",
 'cloud9',
 'clutch gamers',
 'comanche',
 'complexity gaming',
 'crescendo',
 'cyber anji',
 'dd.dota',
 'dt^club',
 'danish bears',
 'darer',
 'detonator',
 'digital chaos',
 'digital chaos.sa',
 'doo wop',
 'double dimension',
 'dragneel',
 'dream gaming',
 'drinkingboys',
 'druidz',
 'duza gaming

In [38]:
#df.to_csv('dob.csv', index=False, encoding='utf-8')

In [19]:
@rate_limited(1, 2)
def requestQuery(title):
    headers = {
    'User-Agent': 'Data for Research',
    'From': 'terthasarit@live.com', 
    'Accept-Encoding': 'gzip'
    }
    r = requests.get(url='http://liquipedia.net/dota2/api.php?action=query&format=json&prop=linkshere&lhnamespace=10&lhlimit=500&titles=Template%3ATeam%2F' + title, headers=headers)
    json = r.json()
    return json

In [22]:
ts = requestQuery('4 friends + chrillee')

In [24]:
ts

{'batchcomplete': '',
 'query': {'normalized': [{'from': 'Template:Team/4 friends   chrillee',
    'to': 'Template:Team/4 friends chrillee'}],
  'pages': {'4929': {'linkshere': [{'ns': 10,
      'pageid': 4938,
      'redirect': '',
      'title': 'Template:Team/4fc'},
     {'ns': 10,
      'pageid': 23377,
      'redirect': '',
      'title': 'Template:Team/4 friends + chrillee'}],
    'ns': 10,
    'pageid': 4929,
    'title': 'Template:Team/4 friends chrillee'}}}}

In [47]:
AlternateIdDict = {}
json = requestQuery('4 friends + chrillee')
for pageid, ldata in json.get('query').get('pages').items():
    mainid = easyText(ldata['title'])
    try:
        for altid in ldata['linkshere']:
            AlternateIdDict[re.search(r'Template:Team/(.+)', altid['title'])[1]] = '4 friends + chrillee'
    except KeyError:
        AlternateIdDict['4 friends + chrillee'] = '4 friends + chrillee'

In [48]:
AlternateIdDict

{'4 friends + chrillee': '4 friends + chrillee', '4fc': '4 friends + chrillee'}

In [61]:
teamShort = {}

In [51]:
idlcs[:10]

['(monkey) business',
 'aspera',
 '1st.vn',
 '4 anchors + sea captain',
 '4 clover & lepricon',
 '4 friends + chrillee',
 '4 vikings + dane',
 '4 protect five',
 '5jungz',
 '90s gaming']

In [102]:
%%time
for title in slash:
    json = requestQuery(title)
    for pageid, ldata in json.get('query').get('pages').items():
        mainid = easyText(ldata['title'])
        try:
            for altid in ldata['linkshere']:
                try:
                    teamShort[re.search(r'Template:Team/(.+)', altid['title'])[1]] = title
                except TypeError:
                    print(title + ' NoneType')
                    continue
        except KeyError:
            teamShort[title] = title


Wall time: 30.6 s


In [108]:
teamShort

{'.aspera.': 'aspera',
 '1stvn': '1st.vn',
 '4 anchors + sea captain': '4 anchors + sea captain',
 '4 clover & lepricon': '4 clover & lepricon',
 '4 friends + chrillee': '4 friends + chrillee',
 '4 vikings + dane': '4 vikings + dane',
 '4fc': '4 friends + chrillee',
 '4p5': '4 protect five',
 '5jungz': '5jungz',
 '90s': '90s gaming',
 'Flip.sid3 Tactics': 'flipsid3 tactics',
 'ROOT Gaming': 'root gaming',
 'aces': 'aces gaming',
 'acion arena': 'acion arena',
 'ad finem': 'ad finem',
 'alternate': 'alternate attax',
 'ancient warriors': 'ancient warriors',
 'animal kingdom': 'kingdom dota',
 'anji': 'cyber anji',
 'anzhi': 'cyber anji',
 'arcade': 'arcade.powerrangers',
 'arcade esports': 'arcade.powerrangers',
 'arcade.power rangers': 'arcade.powerrangers',
 'arcade.pr': 'arcade.powerrangers',
 'arcanys gaming': 'arcanys gaming',
 'archon': 'team archon',
 'atn': 'alternate attax',
 'avalon': 'avalon club',
 'b)': 'b)ears',
 'bad english': 'team bad english',
 'balrogs': 'balrogs',
 '

In [97]:
slash = []

In [100]:
for t in idlcs:
    if '/' in t:
        slash.append(re.search(r'(.+)/(.+)', t)[1])

In [101]:
slash

['elements one',
 'elements pro',
 'evernova',
 'evernovas',
 'orange.taring',
 'team bazaar.youth',
 'dk scuderia',
 'team dk',
 'elements pro gaming',
 'elements pro gaming',
 'evernova',
 'evernova',
 'orange esports',
 'team bazaar',
 'team dk',
 'team dk']

In [88]:
idlcs

['(monkey) business',
 'aspera',
 '1st.vn',
 '4 anchors + sea captain',
 '4 clover & lepricon',
 '4 friends + chrillee',
 '4 vikings + dane',
 '4 protect five',
 '5jungz',
 '90s gaming',
 'alternate attax',
 'aces gaming',
 'acion arena',
 'ad finem',
 'alliance',
 'ancient warriors',
 'animal planet',
 'arcade.powerrangers',
 'arcanys gaming',
 'avalon club',
 'b)ears',
 'boom id',
 'balrogs',
 'basically unknown',
 'beyond infinity',
 'big god',
 'big red machine',
 'bravado gaming',
 'brave heart',
 'broodmothers',
 'burden united',
 'cavalry',
 'cdec avenger',
 'cdec gaming',
 'cdec youth',
 'cis rejects',
 'cookiss',
 "can't say wips",
 'cascade',
 "summer's rift",
 "click 'n search",
 'cloud9',
 'clutch gamers',
 'comanche',
 'complexity gaming',
 'crescendo',
 'cyber anji',
 'dd.dota',
 'dt^club',
 'danish bears',
 'darer',
 'detonator',
 'digital chaos',
 'digital chaos.sa',
 'doo wop',
 'double dimension',
 'dragneel',
 'dream gaming',
 'drinkingboys',
 'druidz',
 'duza gaming

In [104]:
save_obj(teamShort, 'dict/teamShort')

In [ ]:
 json.get('query').get('results')['N0tail']

In [ ]:
len(df)

In [ ]:
def AlternateIdDict(AlternateIdDict, json):
    for pageid, ldata in json.get('query').get('pages').items():
        mainid = easyText(ldata['title'])
        try:
            AlternateIdDict[mainid] = mainid
            for altid in ldata['redirects']:
                AlternateIdDict[easyText(altid['title'])] = mainid
        except KeyError:
            AlternateIdDict[mainid] = mainid
    return AlternateIdDict

In [ ]:
df['idlow'] = df.id

In [ ]:
df.idlow = df.idlow.str.replace(r'\W+', '').str.lower()

In [ ]:
df[df['player'] == 'kpii']

In [ ]:
df.to_csv('idanddob.csv', index=False)

In [111]:
teamShort['dk']

'team dk'

In [112]:
json

{'batchcomplete': '',
 'query': {'pages': {'2395': {'linkshere': [{'ns': 10,
      'pageid': 2396,
      'redirect': '',
      'title': 'Template:Team/dk'},
     {'ns': 10,
      'pageid': 46532,
      'redirect': '',
      'title': 'Template:Team/team dk/team dk'}],
    'ns': 10,
    'pageid': 2395,
    'title': 'Template:Team/team dk'}}}}